# TP2: Machine Learning

In [ ]:
import pandas as pd

# import ML packages
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn import linear_model
from sklearn import preprocessing

# import Plotting pckgs
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
trainingSet = pd.read_csv('../CSVs/improved_trip_train.csv')
testingSet = pd.read_csv('../CSVs/improved_trip_test.csv')
trainingSet.dtypes

In [ ]:
trainingSet.head()

In [ ]:
trainingSet.shape

In [ ]:
testingSet.head()

In [ ]:
trainingSet.duration.mean()

### Let's study correlation between data

In [ ]:
trainingSet.corr()['duration']

We can see that the duration is most correlated with:
    - start_dayOfWeek     0.018351
    - end_dayOfWeek       0.013753
    - end_hour            0.009626
    - start_station_id   -0.007001

In [ ]:
columns = trainingSet.columns.tolist()
columns

In [ ]:
trainingSet.dtypes

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [ ]:
trainingSet.shape

In [ ]:
# Train the model using the training sets
regr.fit(trainingSet[['start_dayOfWeek', 'end_dayOfWeek', 'end_hour', 'start_station_id']], trainingSet.duration)

In [ ]:
prediction = regr.predict(testingSet[['start_dayOfWeek', 'end_dayOfWeek', 'end_hour', 'start_station_id']])

In [ ]:
prediction